In [24]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [3]:
outputDir = 'scraped_json//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('sourceFiles.json', "r").read())

## Load the file schema - THIS ISN'T BEING USED :S
sourceSchema = json.loads(open('sourceSchema.json', "r").read())

## Load the id config
idConfig = json.loads(open('idConfig.json', "r").read())

In [4]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dicti
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [5]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
for key in sourceFiles.keys():
    print(key)
    vars()[key] = mergeSourceFiles(key, outputDir, sourceFiles)
    
    

sports247
rivals
cfbRef
allConf
allAmerican
nflData
ncaa


In [6]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state") 

In [7]:
#create a universal identifier
#CHANGE - this is not going to be a list
#CFB-Ref reports it in chronological order so we will join based on position 1

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [8]:
createNewID(idConfig['sports247'], sports247, field_agg)
createNewID(idConfig['rivals'], rivals, field_agg)
createNewID(idConfig['cfbRef'], cfbRef, field_agg)
createNewID(idConfig['allConf'], allConf, field_agg)
createNewID(idConfig['allAmerican'], allAmerican, field_agg)
createNewID(idConfig['nflData'], nflData, field_agg)
createNewID(idConfig['ncaa'], ncaa, field_agg)

In [26]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)
df_ncaa = pandas.DataFrame(ncaa)

dfs= [df_sports247, df_rivals, df_cfbRef, df_allConf, df_allAmerican, df_nflData, df_ncaa]

In [48]:
## I have no how it works
# BUT IT IS BEAUTIFUL
## src: https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns
df_final = reduce(lambda left,right: pandas.merge(left,right,how="left", on='ID'), dfs)
df_final.head()

,school_x,year_x,playerName_x,highSchool_x,city_x,state_x,position_x,height_x,weight_x,compRating,...,all_pros_first_team,pro_bowls,years_as_primary_starter,g,college_id,name,team_y,gamesPlayed_y,gamesStarted,year_y
0,nebraska,2012,paulthurston,Arvada West,Arvada,CO,OT,77,275,0.9407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nebraska,2012,gregmcmullen,Archbishop Hoban,Akron,OH,SDE,77,255,0.9099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,WR,73,190,0.9134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,WR,73,190,0.9134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nebraska,2012,imanicross,North Hall,Gainesville,GA,RB,73,222,0.9093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
final = df_final.to_csv()

In [50]:
with open("pdmerge.csv", "w", encoding="utf-8") as write_file:
                write_file.write(df_final.to_csv())

In [20]:
left_merged = pandas.merge(df_sports247, df_rivals, suffixes=["","_rivals"],
                        how="left", on=["ID", "ID"])
left_merged.head()
left_merged.shape

(17403, 33)

In [23]:
left_merged.columns

Index(['school', 'year', 'playerName', 'highSchool', 'city', 'state',
       'position', 'height', 'weight', 'compRating', 'compStars',
       'nationalRank', 'positionRank', 'stateRank', '247Rating', '247Stars',
       '247positionRank', '247stateRank', 'ID', '247nationalRank',
       'school_rivals', 'year_rivals', 'playerName_rivals', 'city_rivals',
       'state_rivals', 'highSchool_rivals', 'position_rivals', 'height_rivals',
       'weight_rivals', 'stars', 'nationalRank_rivals', 'positionRank_rivals',
       'stateRank_rivals'],
      dtype='object')